In [1]:
import pandas as pd
import re
import numpy as np
import requests


In [2]:
all_bills = pd.read_csv("all_bills.csv")
bills_with_mgl = pd.read_csv("12billswithmgl.csv")
bills_with_mgl.drop(columns = ['Unnamed: 0.1','Unnamed: 0'], inplace=True)
bills_with_mgl
bills_with_mgl_12 = pd.read_csv("12_bills_with_mgl_more_sections.csv")
# bills_with_mgl["DocumentText"][9]
all_bills

,Title,BillNumber,DocketNumber,GeneralCourtNumber,PrimarySponsor,Cosponsors,JointSponsor,BillHistory,LegislationTypeName,Pinslip,DocumentText,EmergencyPreamble,RollCalls,Attachments,CommitteeRecommendations,Amendments
0,An Act making appropriations for the Fiscal Ye...,H1,HD1,193.0,"{'Id': 'GOV7', 'Name': 'Maura T. Healey', 'Typ...",[],NaN,http://malegislature.gov/api/GeneralCourts/193...,Bill,NaN,NaN,NaN,[],[],[],[]
1,An Act relative to the recovery of overearnings,H10,HD10,193.0,"{'Id': None, 'Name': 'Public Employee Retireme...",[],NaN,http://malegislature.gov/api/GeneralCourts/193...,Bill,So much of the recommendations of the Public E...,\tParagraph (c) of Section 91 of chapter 32 of...,NaN,[],[],[],[]
2,An Act relative to buffer zones on land under ...,H100,HD992,193.0,"{'Id': 'PAS1', 'Name': 'Paul A. Schmid, III', ...","[{'Id': 'PAS1', 'Name': 'Paul A. Schmid, III',...",NaN,http://malegislature.gov/api/GeneralCourts/193...,Bill,"By Representative Schmid of Westport, a petiti...",\tSECTION 1. Chapter 20 Section 23 of the Gene...,NaN,[],[],[],[]
3,An Act providing living organ donor protections,H1000,HD941,193.0,"{'Id': 'WJD1', 'Name': 'William J. Driscoll, J...","[{'Id': 'WJD1', 'Name': 'William J. Driscoll, ...",NaN,http://malegislature.gov/api/GeneralCourts/193...,Bill,"By Representative Driscoll of Milton, a petiti...",\tSECTION 1. Chapter 26 of the General Laws is...,NaN,[],[],[],[]
4,An Act relative to step therapy and in vitro f...,H1001,HD952,193.0,"{'Id': 'WJD1', 'Name': 'William J. Driscoll, J...","[{'Id': 'WJD1', 'Name': 'William J. Driscoll, ...",NaN,http://malegislature.gov/api/GeneralCourts/193...,Bill,"By Representative Driscoll of Milton, a petiti...",\tSECTION 1. Chapter 32A of the General Laws i...,NaN,[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6591,An Act to modernize the roles and responsibili...,S995,SD223,193.0,"{'Id': 'A_G0', 'Name': 'Adam Gomez', 'Type': 1...","[{'Id': 'A_G0', 'Name': 'Adam Gomez', 'Type': ...",NaN,http://malegislature.gov/api/GeneralCourts/193...,Bill,"By Mr. Gomez, a petition (accompanied by bill,...","\tSECTION 1. Section 3 of chapter 258B, as app...",NaN,[],[],[],[]
6592,An Act to reform the hate crime statutes,S996,SD459,193.0,"{'Id': 'A_G0', 'Name': 'Adam Gomez', 'Type': 1...","[{'Id': 'A_G0', 'Name': 'Adam Gomez', 'Type': ...",NaN,http://malegislature.gov/api/GeneralCourts/193...,Bill,"By Mr. Gomez, a petition (accompanied by bill,...",\tSECTION 1. Section 222 of chapter 6 of the G...,"Whereas, The deferred operation of this act wo...",[],[],[],[]
6593,An Act relative to Massachusetts state soverei...,S997,SD506,193.0,"{'Id': 'A_G0', 'Name': 'Adam Gomez', 'Type': 1...","[{'Id': 'A_G0', 'Name': 'Adam Gomez', 'Type': ...",NaN,http://malegislature.gov/api/GeneralCourts/193...,Bill,"By Mr. Gomez, a petition (accompanied by bill,...","\tSECTION 1. Chapter 12 of the General Laws, a...",NaN,[],[],[],[]
6594,An Act to remove collateral consequences and p...,S998,SD744,193.0,"{'Id': 'A_G0', 'Name': 'Adam Gomez', 'Type': 1...","[{'Id': 'A_G0', 'Name': 'Adam Gomez', 'Type': ...",NaN,http://malegislature.gov/api/GeneralCourts/193...,Bill,"By Mr. Gomez, a petition (accompanied by bill,...","\tSECTION 1. Section 172(a)(3) of chapter 6, a...",NaN,[],[],[],[]


In [3]:
def get_sections(bill):
    regex = ""
    chapter = ""
    section = ""
    
    
    check_list = ["section", "chapter"]
    # regex = re.findall(r'(Section|section)(\s+\d+[a-zA-Z]|\s+\d+)\s+(of)\s(Chapter|chapter)(\s+\d+[a-zA-Z]|\s\d+)', doc)
    # regex = re.findall(r'(Section|section)(\s+\d+[a-zA-Z]|\s+\d+)\s+(of|of\ssaid)\s(Chapter|chapter)(\s+\d+[a-zA-Z]|\s\d+)|(Section|section|Chapter|chapter)(\s+\d+[a-zA-Z]|\s\d+)', doc)
    regex = re.findall(r'(section)(\s+\d+[a-zA-Z]+|\s+\d+)\s+(of|of\ssaid)\s(chapter)(\s+\d+[a-zA-Z]+|\s\d+)|(section|chapter)(\s+\d+[a-zA-Z]+|\s\d+)',str(bill), re.IGNORECASE)
            
    lists_with_both =[]
    current_chapter = ""

    #iterate over regex list that contains both chapters and sections
    for x in regex:  
        items = []
        for item in x:
            item = item.casefold()
            items.append(item) 
        if all(name in items for name in check_list):
            for i, j in enumerate(x):
        
                if j.casefold() == "section":
                    section = x[i+1].strip()
        
                if j.casefold() == "chapter":
                    chapter = x[i+1].strip()
                    current_chapter = chapter #save current chapter in order to use for sections mentioned later
                    
                    #add chapter and section to the list
                    list_with_both = [chapter, section]
                    
                    #only keep new chapter and section combo
                    if list_with_both not in lists_with_both:
                        lists_with_both.append(list_with_both)

        else:
            #iterate over list that contains only sections or chapters
            for i, j in enumerate(x):
                if j == "SECTION": #ignore SECTION with caps as it indicates sections from the bills and not the MGL
                    continue
                if j == "":
                    continue
                
                else:
                    if j.casefold() == "chapter":
                        current_chapter = x[i+1].strip() #keep track of current chapter
                    
                    if j.casefold() == "section":
                        if x[i+1] == "":
                            continue
                        else:
                            section = x[i+1].strip()
                        
                        list_with_both = [current_chapter, section]
                        if list_with_both not in lists_with_both:
                            lists_with_both.append(list_with_both)
                            
    return lists_with_both

In [4]:
all_bills ["Reference List"] = all_bills["DocumentText"].apply(get_sections) #apply function to get MGL references for chapters and sections
mgl_ref_df = all_bills[["Title", "BillNumber", "DocketNumber", "DocumentText", "Reference List"]]

In [5]:
len(mgl_ref_df[mgl_ref_df["Reference List"].str.len() == 0]) # count number of empty lists (includes both empty bills and bills with no chapter sections extracted)

792

In [6]:
empty_lists = mgl_ref_df[mgl_ref_df["Reference List"].str.len() == 0]
# empty_lists["Origina Index"] = empty_lists.index.values
empty_lists.reset_index(inplace=True)
# empty_lists['Original-Index'] == empty_lists.index
# empty_lists["DocumentText"][726]
# empty_lists.reset_index(inplace = True)

count_chapter = 0
count_section = 0
chapter = []
section = []
for idx, doc in empty_lists["DocumentText"].items():
    # print(idx, string)
    
    word_list = str(doc).split(" ")
    i = 0
    for word in word_list:
        if word.casefold() == "section":
            this_word = word_list[i]
            
            count_chapter = 1 + count_chapter
            

            chap_idx = ["Chapter", idx, count_chapter ]
            chapter.append(chap_idx)
            # print(word, idx)
        if word.casefold() == "chapter":
            # print("1", this_word)
            i = (1+i)%len(word_list)
            next_word = word_list[i]
            print(idx, "2", next_word)

            count_section = 1 + count_section
            sec_idx = ["Section", idx, count_section]
            section.append(sec_idx)
            
        if word == "SECTION":
            continue
        else:
            chap_idx = ["Chapter", idx, count_chapter]
            sec_idx = ["Section", idx, count_section]
        
        
        
       

# mgl_ref_df["Reference List"]
# empty_lists["DocumentText"][2]
# # section

3 2 1.
8 2 chapter
9 2 firefighter
9 2 employed
15 2 175
16 2 medications
18 2 (a)
25 2 there
30 2 1.
31 2 1.
32 2 there
34 2 21.
34 2 Chapter
38 2 1.
38 2 Chapter
40 2 the
40 2 many
40 2 Massachusetts
42 2 1.
43 2 1. 
46 2 1.
46 2 Subsection
46 2 (b)
52 2 1.
55 2 sections
61 2 That
64 2 1.
64 2 There
72 2 1.
73 2 1.
73 2 There
73 2 shall
82 2 1.
84 2 any
84 2 special
87 2 1.
92 2 that
102 2 1.
102 2 Notwithstanding
103 2 1.
107 2 1.
107 2 Notwithstanding
116 2 XXX.
116 2 Emergency
116 2 Medical
116 2 Services.
116 2 Notwithstanding
116 2 the
117 2 ##.
142 2 Chapter
146 2 1.
147 2 1.
148 2 chapter
150 2 1.
153 2 the
153 2 provisions
156 2 1.
159 2 1.
164 2 sections
164 2 65A,
167 2 chapter
168 2 1.
168 2 Section
170 2 any
170 2 general
170 2 or
177 2 would
178 2 1.
183 2 1.
185 2 Reforming
185 2 the
185 2 Chapter
185 2 61/61A
185 2 Forest
185 2 &
185 2 Farm
185 2 Land
185 2 Tax
185 2 Programs
185 2 to
185 2 Encourage
187 2 23K
188 2 23K
189 2 1.
192 2 constitution
193 2 commissioner
19

In [7]:
import urllib3
urllib3.disable_warnings()
def make_api_call(item):
    result = """"""
    
    try:
        # unpack section and chapter for example: ['2', '15D']
        chapter, section = item
        link = f'https://malegislature.gov/api/Chapters/{chapter}/Sections/{section}'
        r = requests.get(link, verify=False)
        r = r.json()

        # fields to extract
        result =  r.get("Text", np.nan)
        return result
    except:
        pass

In [3]:
# sample = ref_df['Reference List']

def get_mgl(sample):
    # print(df.index)
    print("Chapter-Sections", sample)
    
# Storing and printing each pair
    formatted_data = []
    result = """"""
    if len(str(sample)) == 0:
        return
    if sample == None:
        return
    if str(sample) == "nan":
        return
    
    for pair in sample:
        
        if len(pair) == 0:
            continue
        else:
            string = make_api_call(pair)
            if string == None:
                continue
            if string == np.nan:
                continue
            if str(string) == "nan":
                continue
            if str(string) == "":
                continue
            else:
                result += string
        formatted_data.append(result)
        
    return formatted_data

 

In [19]:
# mgl_ref_df['MGL Ref'] = mgl_ref_df["Reference List"].apply(get_mgl)
mgl_ref_df['MGL Ref']= mgl_ref_df["Reference List"].apply(get_mgl)#takes over 2.5 hour to run



Chapter-Sections []
Chapter-Sections [['32', '91']]
Chapter-Sections [['20', '23']]
Chapter-Sections [['26', '8L'], ['26', '8M'], ['175', '47OO'], ['176A', '39'], ['176B', '26'], ['176G', '34'], ['111', '243'], ['175', '108N'], ['175', '108O'], ['175', '120F'], ['175', '120G'], ['176U', '3']]
Chapter-Sections [['32A', '17R'], ['32A', '17S'], ['118E', '10N'], ['118E', '10O'], ['175', '47H'], ['176A', '8K'], ['176B', '4J']]
Chapter-Sections [['32A', '23'], ['112', '196'], ['176G', '4N']]
Chapter-Sections [['32B', '19'], ['32B', '23']]
Chapter-Sections [['32A', '17R'], ['32A', '17S'], ['175', '47QQ'], ['175', '47RR'], ['175', '108'], ['175', '110'], ['176A', '8QQ'], ['176A', '8RR'], ['176B', '4QQ'], ['176B', '4RR'], ['176G', '4GG'], ['176G', '4II']]
Chapter-Sections [['26', '8G'], ['26', '142M']]
Chapter-Sections [['90', '34A'], ['90', '34O']]
Chapter-Sections [['32A', '17R'], ['32A', '17S'], ['176O', '1'], ['111O', '1'], ['111O', '2'], ['118E', '10N'], ['118E', '10O'], ['175', '47PP'], [

/var/folders/2p/c1tbzy690j51h1d7y06z278m0000gn/T/ipykernel_1338/2611607304.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mgl_ref_df['MGL Ref']= mgl_ref_df["Reference List"].apply(get_mgl)#takes over 2.5 hour to run


In [13]:
len(mgl_ref_df[mgl_ref_df["MGL Ref"].str.len() == 0])

1313

In [22]:

# mgl_ref_df['combined_MGL'] = join_strings = lambda x: ' '.join(x)
# Apply the lambda function to the specified column
mgl_ref_df['Combined_MGL'] = mgl_ref_df['MGL Ref'].apply(lambda x: ' '.join(x) if len(x) != 0 else np.NaN)
mgl_ref_df['Combined_MGL'] 

# [(lambda x:True if x>0 and x< 7 else False)(i) for i in myList]
# mgl_ref_df['combined_MGL']


/var/folders/2p/c1tbzy690j51h1d7y06z278m0000gn/T/ipykernel_1338/1691864776.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mgl_ref_df['Combined_MGL'] = mgl_ref_df['MGL Ref'].apply(lambda x: ' '.join(x) if len(x) != 0 else np.NaN)


0                                                     NaN
1       Section 91. (a) No person while receiving a pe...
2       Section 23. (a) The secretary of environmental...
3       Section 8L. In regard to any carrier licensed ...
4       [Text of section effective as applied to contr...
                              ...                        
6591    Section 3. To provide victims a meaningful rol...
6592    Section 33. The colonel shall promulgate regul...
6593    Section 5O. Nothing in sections 5B to 5M, incl...
6594    Section 172. (a) The department shall maintain...
6595    Section 127I. Upon the filing of a petition to...
Name: Combined_MGL, Length: 6596, dtype: object

In [23]:
mgl_ref_df
# new_mgl_df

,Title,BillNumber,DocketNumber,DocumentText,Reference List,MGL Ref,Combined_MGL,NumWords
0,An Act making appropriations for the Fiscal Ye...,H1,HD1,NaN,[],[],NaN,0
1,An Act relative to the recovery of overearnings,H10,HD10,\tParagraph (c) of Section 91 of chapter 32 of...,"[[32, 91]]",[Section 91. (a) No person while receiving a p...,Section 91. (a) No person while receiving a pe...,0
2,An Act relative to buffer zones on land under ...,H100,HD992,\tSECTION 1. Chapter 20 Section 23 of the Gene...,"[[20, 23]]",[Section 23. (a) The secretary of environmenta...,Section 23. (a) The secretary of environmental...,2021
3,An Act providing living organ donor protections,H1000,HD941,\tSECTION 1. Chapter 26 of the General Laws is...,"[[26, 8L], [26, 8M], [175, 47OO], [176A, 39], ...",[Section 8L. In regard to any carrier licensed...,Section 8L. In regard to any carrier licensed ...,6666
4,An Act relative to step therapy and in vitro f...,H1001,HD952,\tSECTION 1. Chapter 32A of the General Laws i...,"[[32A, 17R], [32A, 17S], [118E, 10N], [118E, 1...",[[Text of section effective as applied to cont...,[Text of section effective as applied to contr...,4636
...,...,...,...,...,...,...,...,...
6591,An Act to modernize the roles and responsibili...,S995,SD223,"\tSECTION 1. Section 3 of chapter 258B, as app...","[[258B, 3], [258B, 4], [258B, 6]]",[Section 3. To provide victims a meaningful ro...,Section 3. To provide victims a meaningful rol...,7274
6592,An Act to reform the hate crime statutes,S996,SD459,\tSECTION 1. Section 222 of chapter 6 of the G...,"[[6, 222], [30, 39], [22C, 33], [22C, 34], [26...",[Section 33. The colonel shall promulgate regu...,Section 33. The colonel shall promulgate regul...,2526
6593,An Act relative to Massachusetts state soverei...,S997,SD506,"\tSECTION 1. Chapter 12 of the General Laws, a...","[[12, 5O], [12, 5P], [12, 1357], [12, 5Q]]","[Section 5O. Nothing in sections 5B to 5M, inc...","Section 5O. Nothing in sections 5B to 5M, incl...",33
6594,An Act to remove collateral consequences and p...,S998,SD744,"\tSECTION 1. Section 172(a)(3) of chapter 6, a...","[[, 172], [6, 172], [276, 100A], [276, 100B], ...",[Section 172. (a) The department shall maintai...,Section 172. (a) The department shall maintain...,23527


/var/folders/2p/c1tbzy690j51h1d7y06z278m0000gn/T/ipykernel_1338/1279822735.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mgl_ref_df['NumWords'] = mgl_ref_df['Combined_MGL'].apply(count_words)


In [4]:
mgl_ref_df.to_csv("most_bills_mgl.csv")

NameError: name 'mgl_ref_df' is not defined

In [4]:
mgl_ref = pd.read_csv("most_bills_mgl.csv")
mgl_ref.drop(columns = "Unnamed: 0", inplace=True)
mgl_ref

,Title,BillNumber,DocketNumber,DocumentText,Reference List,MGL Ref,Combined_MGL,NumWords
0,An Act making appropriations for the Fiscal Ye...,H1,HD1,NaN,[],[],NaN,0
1,An Act relative to the recovery of overearnings,H10,HD10,\tParagraph (c) of Section 91 of chapter 32 of...,"[['32', '91']]","[""Section 91. (a) No person while receiving a ...",Section 91. (a) No person while receiving a pe...,1605
2,An Act relative to buffer zones on land under ...,H100,HD992,\tSECTION 1. Chapter 20 Section 23 of the Gene...,"[['20', '23']]","[""Section 23. (a) The secretary of environment...",Section 23. (a) The secretary of environmental...,2021
3,An Act providing living organ donor protections,H1000,HD941,\tSECTION 1. Chapter 26 of the General Laws is...,"[['26', '8L'], ['26', '8M'], ['175', '47OO'], ...","[""Section 8L. In regard to any carrier license...",Section 8L. In regard to any carrier licensed ...,11216
4,An Act relative to step therapy and in vitro f...,H1001,HD952,\tSECTION 1. Chapter 32A of the General Laws i...,"[['32A', '17R'], ['32A', '17S'], ['118E', '10N...",['[Text of section effective as applied to con...,[Text of section effective as applied to contr...,6101
...,...,...,...,...,...,...,...,...
6591,An Act to modernize the roles and responsibili...,S995,SD223,"\tSECTION 1. Section 3 of chapter 258B, as app...","[['258B', '3'], ['258B', '4'], ['258B', '6']]","[""Section 3. To provide victims a meaningful r...",Section 3. To provide victims a meaningful rol...,7274
6592,An Act to reform the hate crime statutes,S996,SD459,\tSECTION 1. Section 222 of chapter 6 of the G...,"[['6', '222'], ['30', '39'], ['22C', '33'], ['...",['Section 33. The colonel shall promulgate reg...,Section 33. The colonel shall promulgate regul...,2526
6593,An Act relative to Massachusetts state soverei...,S997,SD506,"\tSECTION 1. Chapter 12 of the General Laws, a...","[['12', '5O'], ['12', '5P'], ['12', '1357'], [...","['Section 5O. Nothing in sections 5B to 5M, in...","Section 5O. Nothing in sections 5B to 5M, incl...",33
6594,An Act to remove collateral consequences and p...,S998,SD744,"\tSECTION 1. Section 172(a)(3) of chapter 6, a...","[['', '172'], ['6', '172'], ['276', '100A'], [...","[""Section 172. (a) The department shall mainta...",Section 172. (a) The department shall maintain...,23527


In [5]:
bill_info = mgl_ref["BillNumber"] + ": " + mgl_ref["Title"]

In [6]:
mgl_ref.insert(0, 'Bill Info', bill_info)


In [8]:
mgl_ref.to_csv("most_bills_mgl.csv")

In [25]:
mgl_ref.to_parquet("all_bills_with_mgl.pq")

In [24]:
bill_info = mgl_ref["Bill Info"].values
for bills in bill_info:
    # print(type(bills))
    bill = bills.split(":")
    # print(bill[0])
    print(bill)

print(bill_info)
bills_to_select = {s.split(": ")[0]: s.split(": ")[1] for s in bill_info}
print(bills_to_select)

['H1', ' An Act making appropriations for the Fiscal Year 2024 for the maintenance of the departments, boards, commissions, institutions, and certain activities of the Commonwealth, for interest, sinking fund, and serial bond requirements, and for certain permanent improvements']
['H10', ' An Act relative to the recovery of overearnings']
['H100', ' An Act relative to buffer zones on land under the APR Program']
['H1000', ' An Act providing living organ donor protections']
['H1001', ' An Act relative to step therapy and in vitro fertilization insurance coverage']
['H1002', " An Act relative to children's auditory health"]
['H1003', ' An Act further regulating the withdrawal of local subscribers from the commission.\r\n']
['H1004', ' An Act relative to cognitive rehabilitation for individuals with an acquired brain injury']
['H1005', ' An Act relative to the licensure of automobile damage appraisers ']
['H1006', ' An Act relative to the compulsory automobile insurance limits']
['H1007',

AttributeError: 'float' object has no attribute 'split'